In [8]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf  # To calculate the logit regression
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
import seaborn as sns
import math

import statsmodels.stats.proportion as ssprop  # The standardized effects sized
import statsmodels.stats.power as ssp   # For traditional power analysis

## Business Case

This is the business case of a fictional company, AirCnC. The management of AirCnC has determined that adding a "1-click booking" button will significantly increase the company's booking rate. In this business case, there will be 3 steps in conducting the experiment:

1. Randomization the assignment of the treatment and control to the user ID
2. Determining the sample size of the experiment

And let's say, if we already conducted the experiment and have the data, the last one is:

3. Analyzing the result using traditional regression and bootstrap simulation

This business case is taken from **Behavioral Data Analysis with R and Python (2021) written by Florent Buisson**.

### Theory of Change

Implementing a **1-click booking button** (intervention) will help us achieve **higher revenue**, as measured by **booking probability**, through **a reduction in the duration of the booking process**.

### Data

There are two CSV files for this case: "chap8-historical_data.csv" and "chap8-experimental_data.csv". The historical data will be used to assign the user ID whether they will fall in treatment or control group. Whereas for experimental, it's a hypothetical data when we already have the data and used to run the experiment and analysis. The column contains:

- **Gender** : Categorical, “male”/ “female”
- **Period** : Month index, 1-32 in historical data, 33 in experimental data
- **Seasonality** : Annual seasonality, between 0 and 1
- **Month** : Month of year, 1-12
- **Booked** : Binary 0/1, target variable
- **Oneclick** : Binary 0/1, experimental treatment (only in experimental data)

Check the code below:

In [9]:
df_hist = pd.read_csv("chap8-historical_data.csv")
df_exp = pd.read_csv("chap8-experimental_data.csv")

In [10]:
df_hist_len = df_hist.shape[0]

print("Historical data:")
display(df_hist)
print(f"There are {df_hist_len} rows of data")

Historical data:


,age,gender,period,seasonality,month,booked
0,34,male,1,0.749993,2,0
1,39,male,1,0.749993,2,0
2,42,male,1,0.749993,2,0
3,33,male,1,0.749993,2,0
4,32,male,1,0.749993,2,1
...,...,...,...,...,...,...
319752,47,female,32,0.067111,9,0
319753,38,female,32,0.067111,9,0
319754,46,male,32,0.067111,9,0
319755,47,male,32,0.067111,9,0


There are 319757 rows of data


In [11]:
df_exp_len = df_exp.shape[0]

print("Experiment data:")
display(df_exp)
print(f"There are {df_exp_len} rows of data")

Experiment data:


,age,gender,period,seasonality,month,booked,oneclick
0,47,male,33,6.492170e-08,10,0,1
1,49,male,33,6.492170e-08,10,0,1
2,37,male,33,6.492170e-08,10,0,0
3,44,male,33,6.492170e-08,10,0,1
4,45,male,33,6.492170e-08,10,0,1
...,...,...,...,...,...,...,...
40155,38,male,36,4.997220e-01,1,1,1
40156,34,male,36,4.997220e-01,1,1,0
40157,28,male,36,4.997220e-01,1,1,1
40158,36,male,36,4.997220e-01,1,0,0


There are 40160 rows of data


In this data, we're interested to investigate the correlation between **"oneclick"** variable to our target variable, **"booked"**. We also made a hypothesis that **"age"** and **"gender"** might affect/influence the path of "oneclick" variable to the dependent variable.

As our dependent variable is a binary variable, we're going to use one of the Generalized Linear Model, the **logit regression**.

## Determining Random Assignment and Sample Size/Power

The following step is to decide how we will conduct the random assignment and the amount of the sample size we will require after developing and validating the theory of change for your experiment.

### Random Assignment

Assuming we are not using a program that handles this for us, we can simply do it in Python by the code below:

In [12]:
def random_assignment(k):  ## k is the number of groups we want (including control)
    ra_assgnt = np.random.uniform(0,1,1)  ## randomize number between 0 and 1 
    ra_group = "control" if ra_assgnt <= 1/k else "treatment"  ## if 'assnt' below 1/K, then it's control, else it's going to fall in 'treatment'
    return ra_group

Let's say we're going to apply this in our historical data, just for a simulation:

In [13]:
df_hist_sim = df_hist  # create dataframe for simulation

for i in range(len(df_hist_sim)):  
    df_hist_sim.loc[i, 'group'] = random_assignment(2)  # call the random assignment function from previous code

df_hist_sim

,age,gender,period,seasonality,month,booked,group
0,34,male,1,0.749993,2,0,control
1,39,male,1,0.749993,2,0,control
2,42,male,1,0.749993,2,0,treatment
3,33,male,1,0.749993,2,0,control
4,32,male,1,0.749993,2,1,control
...,...,...,...,...,...,...,...
319752,47,female,32,0.067111,9,0,control
319753,38,female,32,0.067111,9,0,treatment
319754,46,male,32,0.067111,9,0,treatment
319755,47,male,32,0.067111,9,0,control


### Sample Size and Power Analysis (Traditional Approach)

After we define on how to randomize the population, we need to determine our sample size for the experiment. Power Analysis is the method that statisticians have created to determine the necessary sample size for particular statistical tests. We're going to use that for this analysis.

Let's say that the CEO would like to know the effect size of 1%. It would translate into an expected booking rate for our treatment group of 19.5% from of 18.5% in accordance with our previous data. We set **power = 0.8** and **statistical significance = 0.05** for the parameters' standard values.

In [14]:
effect_size = ssprop.proportion_effectsize(0.194, 0.184)  ## to calculate 1% of effect size using
sample_size = ssp.tt_ind_solve_power(effect_size=effect_size, 
                                     alpha=0.05, 
                                     nobs1=None,  # value that we're looking for, set to none
                                     alternative='larger',  ## set to one-tailed test as we don't care the negative or positive area
                                     power=0.8)

print(f"The sample size needed with defined parameter is {sample_size}")

The sample size needed with defined parameter is 18950.818821249803


We got the sample size around 19,000 sample per group, ceiling rounded. What does this mean?

- Let's say, we conducted a substantial number of tests with a total sample size of 38,000.
- In each scenario, if the statistics from the proportions test had a p-value less than 0.05, we would implement the 1-click button.
- The true effect in each of these experiments is 1%.

### Power Analysis without Statistics: Bootstrap Simulations

Our decision rule does not use p-values in Bootstrap simulations. Instead, we apply the treatment if the **Confidence Interval** calculated using the Bootstrap method for the coefficient of interest is greater than a predetermined threshold, often zero. 

We're going first by writing the analysis code, then go through for the simulation

#### Bootstrap Analysis Code

The analysis model that we're going to make is based on 3 functions.

In [15]:
def log_reg_fn(formula, df, treatment_var):
    '''
    formula: write the formula with apostrophe
    df: input the data frame here
    treatment_var: the variable of interest we're investigating
    '''
    model = smf.logit(formula=formula, data=df)
    res = model.fit(disp=0)
    coeff = res.params[treatment_var]  # to generate the coefficient of oneclick
    return coeff

In [16]:
# testing whether the function works or not
log_reg_fn(formula='booked ~ oneclick + age + gender', df=df_exp, treatment_var='oneclick')  

0.15784070750120138

After building the function, we can build the bootstrap function for the logit model. 

In [17]:
def boot_ci_fn(df, b, conf_level): 
    '''
    df: data frame used for bootstrapping
    b: number of bootstrap samples
    conf_level: the confident level of the bootstrap
    '''
    n = len(df)
    coeffs = []
    
    for i in range(b):
        df_sim_boot = df.sample(n=n, replace=True)
        coeff = log_reg_fn(formula='booked ~ oneclick + age + gender', df=df_sim_boot, treatment_var='oneclick')
        coeffs.append(coeff)  ## all list method, it automatically does the inplace, no need to call the variable again
    
    coeffs.sort()  
    start_idx = round(b * (1-conf_level) / 2)  # upper bound for confidence interval
    end_idx = - round(b * (1-conf_level) / 2)  # lower bound for confidence interval
    conf_int = [coeffs[start_idx], coeffs[end_idx]]  # the coefficient value in the bound
    return(conf_int)

In [18]:
%%time
# testing whether the function works or not
# this code is going to run longer, based on the bootstrap sample
boot_ci_fn(df=df_exp, b=100, conf_level=0.9) 

CPU times: user 24.9 s, sys: 1.94 s, total: 26.9 s
Wall time: 19.2 s


[0.07675430217761259, 0.22846357661912722]

Then, for the final, we built the function for the final decision from all of the two previous functions.

In [19]:
def decision_fn(df, b, conf_level): 
    '''
    df: data frame used for bootstrapping
    b: number of bootstrap samples
    conf_level: the confident level of the bootstrap
    '''
    boot_ci = boot_ci_fn(df, b, conf_level)
    
    if boot_ci[0] > 0:
        decision = 1
        interpret = print("Lower bound is " + str(boot_ci[0]) + ", which is higher than 0. We can confidently apply this treatment.")
    else:
        decision = 0
        interpret = print("Lower bound is " + str(boot_ci[0]) + ", which is lower than 0. Do not apply this treatment.")
    return decision, interpret

We're done with all of the function building. Let's try the "decision_fn" function with **100 bootstrap sample** and **confidence level at 90%** with the code below:

In [20]:
%%time
# this code is going to run longer, based on the bootstrap sample
decision, interpret = decision_fn(df=df_exp, b=5, conf_level=0.9)
print(interpret)

Lower bound is 0.09634743452008217, which is higher than 0. We can confidently apply this treatment.
None
CPU times: user 1.2 s, sys: 90 ms, total: 1.29 s
Wall time: 982 ms


#### Power Simulations

After we built the analysis, we'll create the function that will execute a single simulation first. In this simulation, we're going to use the previous model to calculate power size using the bootstrap simulation. For this one, we will use the experiment data that we use before to calculate the power and effect size from several scenarios of bootstrap simulation. This is how the code functions:

In [21]:
def single_sim_fn(Nexp, df, eff_size, b, conf_level):
     '''
     Nexp: number of sample replicated from the original data to be analyzed
     df: Data frame we're going to use
     eff_size: Effect size, well it depends on what is your measurement
     b: number of bootstrap samples
     conf_level: the confident level of the bootstrap
     '''
     # adding predicted probability of booking
     model = smf.logit('booked ~ oneclick + age + gender', data = df_exp)  # change the formula based on your need
     res = model.fit(disp=0)
     df_sim_exp = df.copy()
     df_sim_exp['pred_prob_bkg'] = res.predict(df_sim_exp)
    
     # filtering down to desired sample size
     df_sim_exp = df_sim_exp.sample(Nexp)
    
     # random assignment of experimental groups
     k = 2  # number of group treatment, including the control
     df_sim_exp['oneclick'] = np.where(np.random.uniform(size=Nexp) <= 1/k, 0, 1)  # adding effect to treatment group
     df_sim_exp['pred_prob_bkg'] = np.where(df_sim_exp.oneclick == 1,
                                           df_sim_exp.pred_prob_bkg + eff_size,
                                           df_sim_exp.pred_prob_bkg)
     df_sim_exp['booked'] = np.where(df_sim_exp.pred_prob_bkg >= np.random.uniform(size=Nexp), 1, 0)
     decision, interpret = decision_fn(df_sim_exp, b = b, conf_level = conf_level)  # unpack the decision, just taking the decision number
     return decision

The power function can then be written for a specific effect size and sample size. This function generates experimental data sets periodically and then applies our decision function to them, returning the proportion of them for which we would activate the button:

In [22]:
def power_sim_fn(Nexp, df, eff_size, b, conf_level, Nsim):  # adding Nsim for how many data sets should we simulate
    '''
    Nexp: number of sample replicated from the original data to be analyzed
    df: Data frame we're going to use
    eff_size: Effect size, well it depends on what is your measurement
    b: number of bootstrap samples
    conf_level: the confident level of the bootstrap
    Nsim: Number of how many times this function do the bootstrap simulation
    '''
    power_lst = []
    
    for i in range(Nsim):
        print("starting simulation number", i, "\n") 
        power_lst.append(single_sim_fn(Nexp = Nexp, df = df,
                                       eff_size = eff_size, b = b,
                                       conf_level = conf_level))
    power = np.mean(power_lst)
    return power

After building the functions, we can try to run the Power simulation using the bootstrap method using the code below:

In [23]:
%%time
# this code is going to run longer, based on the bootstrap sample and number of the simulation
# as for example, we will go with 5 bootstrap and N size to make the code goes faster
power_sim_fn(df=df_exp, Nexp = 40000, eff_size=0.01, b=5, conf_level=0.9, Nsim=5)

starting simulation number 0 

Lower bound is 0.23865449643772005, which is higher than 0. We can confidently apply this treatment.
starting simulation number 1 

Lower bound is 0.21533171693458744, which is higher than 0. We can confidently apply this treatment.
starting simulation number 2 

Lower bound is 0.19761820754850687, which is higher than 0. We can confidently apply this treatment.
starting simulation number 3 

Lower bound is 0.22587601134967938, which is higher than 0. We can confidently apply this treatment.
starting simulation number 4 

Lower bound is 0.14064304462835697, which is higher than 0. We can confidently apply this treatment.
CPU times: user 7.67 s, sys: 547 ms, total: 8.22 s
Wall time: 6.73 s


1.0

## Analyzing and Interpreting Experimental Results

We can analyze the results after doing the experiment and gathering the necessary information. The final analyses itself ought to be simple after all the simulated analyses you performed for the power estimation. There are two steps on how to analyze and interpret our result:

1. Run the logistic regression for the coefficient
2. Use the Bootstrap CI method instead of the p-value as the decision tool for significance value

However, in this analysis, there are two things that we have to be really careful that We cannot interpret the coefficient from logistic regression directly. To overcome this, there are two methods that we can employ.

1. Run comparsion between predicted value of the splitted data: one where the variable for the one-click button is set to 1, and to 0 in the other.
2. Using exponential value of the coefficient, and calculate the probability from there.

In [24]:
# look for the traditional coefficient from the logit regression

model = smf.logit('booked ~ oneclick + age + gender', data=df_exp)
res = model.fit()
res.summary()

Optimization terminated successfully.
         Current function value: 0.161220
         Iterations 9


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 booked   No. Observations:                40160
Model:                          Logit   Df Residuals:                    40156
Method:                           MLE   Df Model:                            3
Date:                Fri, 16 Sep 2022   Pseudo R-squ.:                  0.3311
Time:                        17:47:01   Log-Likelihood:                -6474.6
converged:                       True   LL-Null:                       -9679.1
Covariance Type:            nonrobust   LLR p-value:                     0.000
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         11.6928      0.226     51.819      0.000      11.251      12.135
gender[T.male]     0.2542      0.049      5.182      0.000       0.158       0.350
oneclick           0.1578      0.047      3.357      0.001       0.066       0.250
age               -0.3941      0.006    -61.282      0.000      -0.407      -0.381
==================================================================================
"""

In [25]:
# calling the previous bootstrap CI function
boot_ci_fn(df=df_exp, b=100, conf_level=0.9)

[0.09033995134756835, 0.23665123699661353]

With one-click coefficient is equal to 0.1578 and the the CI are above 0, we can say that the **one-click treatment significantly improves the booking rate**. However, as explained before, we can't directly interpret the coefficient. Thus, we're going to use the two methods.

#### Predicted Value Comparison

The idea in here is straight-forward. We create two copies of the experimental data, one with everyone's variable for the one-click button set to 1, and the other with their variable set to 0. We can determine a "average" effect that is fairly similar to the effect you would see if applying the treatment to everyone by comparing the likelihood of booking predicted by our logistic model for these two data sets. It's not scientific, but more understandable.

In [26]:
df_exp['pred_bkg_rate'] = res.predict(df_exp)

In [27]:
def diff_prob_fn(df, model_reg):
    # adding prediction to the model
    df_pred = df
    df_pred['pred_bkg_rate'] = res.predict(df_pred)
    
    # creating new copies for the data
    df_button = df_pred[df_pred['oneclick'] == 1]
    df_no_button = df_pred[df_pred['oneclick'] == 0]
    
    # calculate the mean different between predicted value of both treatments
    diff = df_button['pred_bkg_rate'].mean() - df_no_button['pred_bkg_rate'].mean()
    return diff

# in the book refer to p.200, i found that the code is quite weird:
# diff = button_df.loc[:,'pred_bkg_rate'] - no_button_df.loc[:,'pred_bkg_rate'] return diff.mean()
# diff_prob_fun(exp_data_df, reg_model = log_mod_exp)
# the code above is not applicable as the length between these two data frames is different

diff_prob_fn(df=df_exp, model_reg=model)  # the result is different, but close marginally


0.0033712623850794127

Let's now calculate the coefficient using odds ratio method and compare the coefficient between two of them.

#### Odds Ratio

This one requires you to have a basic understanding about exponential. But to make it simpler, we are going to use the odds ratio from exponential value of our coefficient.

In [28]:
true_coeff_comp = round(diff_prob_fn(df=df_exp, model_reg=model), 3)
true_coeff_odds = round((math.exp(res.params['oneclick']) - 1), 3)

print(f"Using the comparison method, the treatment increase the booking rate by {true_coeff_comp}.")
print(f"Using the odds ratio method, the treatment increase the booking rate by {true_coeff_odds}.")

Using the comparison method, the treatment increase the booking rate by 0.003.
Using the odds ratio method, the treatment increase the booking rate by 0.171.


## Conclusion

In this notebook, it shows how to do the randomization of the sample, power effect simulation, and the analysis itself after we collect the data of the experiment. We can reproduce this notebook for other calculations, not only logistic regression that is shown in this notebook. With a little bit modification, we can use the other models from the regression.

Using the Bootstrap-CI 90% and the bootstrap size of 100 times, we know that the one-click treatment significantly increases the booking rate of the hotel. When we're trying to interpret the coefficient, there's a big different between value between the predicted value and odds ratio method. As a safe railguard, we're going to use the odds ratio method. Thus, th**e one-click treatment significantly increases the booking rate by 17%.**

